## **Part 1** 
**Understand the RL training pipeline in IsaacLab(Environment & Agent Setup, Training process, Performance evaluate)**

#### 1.1. According to the tutorials, if we want to edit the environment configuration, action space, observation space, reward function, or termination condition of the `Isaac-Cartpole-v0` task, which file should we look at, and where is each part located?

All of this component: Environment configuration, action space, observation space, reward function and termination condition can edit in file `cartpole_env_cfg.py` in path of  
`source\extensions\omni.isaac.lab_tasks\omni\isaac\lab_tasks\manager_based\classic\cartpole`

#### 1.2. What are the action space and observation space for an agent defined in the `Isaac-Cartpole-v0` task?

In file `cartpole_env_cfg.py` has config class name ActionsCfg and ObeservationCfg that represent to action space and observation space, so, this is code for space config.  

In [ ]:
# Config class for Action space for the joint effort applied to "silder_to_cart" joint of the cartpole and scale by factor of 100.0

@configclass
class ActionsCfg:
    """Action specifications for the MDP."""

    joint_effort = mdp.JointEffortActionCfg(asset_name="robot", joint_names=["slider_to_cart"], scale=100.0)


# Config class for Observation space for observe joint_pos_rel and joint_vel_rel (relative position and relative velocity)
@configclass
class ObservationsCfg:
    """Observation specifications for the MDP."""

    @configclass
    class PolicyCfg(ObsGroup):
        """Observations for policy group."""

        # observation terms (order preserved)
        joint_pos_rel = ObsTerm(func=mdp.joint_pos_rel)
        joint_vel_rel = ObsTerm(func=mdp.joint_vel_rel)

        def __post_init__(self) -> None:
            self.enable_corruption = False
            self.concatenate_terms = True

    # observation groups
    policy: PolicyCfg = PolicyCfg()

So, in `Isaac_Cartpole-v0` task `action space` is how much force or effort to apply to motor driving the cart sacle by 100.0 and `observation space` is what relative position and relative velocity like when we put encoder to measure position and velocity of motor shaft when it moving.

#### 1.3. How can episodes in the `Isaac-Cartpole-v0` task be terminated?  

In file `cartpole_env_cfg.py` class TerminationsCfg is for terminate term.

In [ ]:
# Config class for Terminate term
@configclass
class TerminationsCfg:
    """Termination terms for the MDP."""

    # (1) Time out
    time_out = DoneTerm(func=mdp.time_out, time_out=True)           # Terminate condition #1 timeout
    # (2) Cart out of bounds
    cart_out_of_bounds = DoneTerm(                                  # Terminate condition #2 cart out of bounds
        func=mdp.joint_pos_out_of_manual_limit,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"]), "bounds": (-3.0, 3.0)},       # Set boundary of cart (Terminate : out of boundary)
    )
    
    
    
# Config class for environment configuration
@configclass
class CartpoleEnvCfg(ManagerBasedRLEnvCfg):
    """Configuration for the cartpole environment."""

    # Scene settings
    scene: CartpoleSceneCfg = CartpoleSceneCfg(num_envs=4096, env_spacing=4.0)
    # Basic settings
    observations: ObservationsCfg = ObservationsCfg()
    actions: ActionsCfg = ActionsCfg()
    events: EventCfg = EventCfg()
    # MDP settings
    rewards: RewardsCfg = RewardsCfg()
    terminations: TerminationsCfg = TerminationsCfg()

    # Post initialization
    def __post_init__(self) -> None:
        """Post initialization."""
        # general settings
        self.decimation = 2
        self.episode_length_s = 5                       # Set episode length (Terminate : timeout term)
        # viewer settings
        self.viewer.eye = (8.0, 0.0, 5.0)
        # simulation settings
        self.sim.dt = 1 / 120
        self.sim.render_interval = self.decimation

In this config class can tell 2 term to terminate the episode. Firstly, timeout in this script is 5 seconds from valiable `self.episode_length_s` in post initialize and secondly cart out of bundary is (-3.0 to 3.0)

#### 1.4. How many reward terms are used to train an agent in the `Isaac-Cartpole-v0` task?

In [ ]:
@configclass
class RewardsCfg:
    """Reward terms for the MDP."""

    # (1) Constant running reward
    alive = RewTerm(func=mdp.is_alive, weight=1.0)
    # (2) Failure penalty
    terminating = RewTerm(func=mdp.is_terminated, weight=-2.0)
    # (3) Primary task: keep pole upright
    pole_pos = RewTerm(
        func=mdp.joint_pos_target_l2,
        weight=-1.0,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"]), "target": 0.0},
    )
    # (4) Shaping tasks: lower cart velocity
    cart_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.01,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"])},
    )
    # (5) Shaping tasks: lower pole angular velocity
    pole_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.005,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"])},
    )

`Isaac-Cartpole-v0` has 5 reward terms for train an agent 1)alive: reward when constant running, 2)terminating: failure penalty, 3)pol_pos: primary task: keep pole upright, 4)cart_vel: shaping task: lower cart velocity, 5)pole_vel: shaping tasks: lower pole angular velocity

## **Part 2**  
**Understand the Isaaclab workflow `Manager-Based RL Environment` for implementation your RL project setup in Isaaclab:Action, Obeservation, Event, Reward, Termination.**  

Adjusting weight of **each reward term** vs default configuration


#### Default Configuration

<!-- Horizontal Top Label -->
<div class="video-top-label" style="text-align: center; font-weight: bold; font-size: 20px; margin-bottom: 20px;">
    Default configuration from IsaacLab
</div>

<!-- Second Section: Videos (1 Column) -->
<div class="video-section" style="display: flex; flex-direction: column; gap: 10px;">
    <!-- Row 1 Label -->
    <div class="video-label" style="text-align: center; font-weight: bold;">Train step 2000</div>
    <video src="videos/default_2000.mp4" controls style="width: 500px; height: auto; margin: 0 auto;"></video>
    <!-- Row 2 Label -->
    <div class="video-label" style="text-align: center; font-weight: bold;">Train step 8000</div>
    <video src="videos/default_8000.mp4" controls style="width: 500px; height: auto; margin: 0 auto;"></video>
    <!-- Row 3 Label -->
    <div class="video-label" style="text-align: center; font-weight: bold;">Train step 14000</div>
    <video src="videos/default_14000.mp4" controls style="width: 500px; height: auto; margin: 0 auto;"></video>
</div>


#### **Reward #1 : alive** (Constant running reward)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting alive (Reward) </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/alive_0.png" alt="Image 1">
            <img src="images/alive_1.png" alt="Image 2">
            <img src="images/alive_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent alive weight = 0.5, Right represent alive weight = 1.5 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/alive_05_2000.mp4" controls></video>
            <video src="videos/alive_15_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/alive_05_8000.mp4" controls></video>
            <video src="videos/alive_15_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/alive_05_14000.mp4" controls></video>
            <video src="videos/alive_15_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### **Reward #2 : terminating** (Failure penalty)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting termination (Pernalty term) </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/terminate_0.png" alt="Image 1">
            <img src="images/terminate_1.png" alt="Image 2">
            <img src="images/terminate_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent terminating weight = -2.5, Right represent terminating weight = -1.5 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/terminate_25_2000.mp4" controls></video>
            <video src="videos/terminate_15_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/terminate_25_8000.mp4" controls></video>
            <video src="videos/terminate_15_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/terminate_25_14000.mp4" controls></video>
            <video src="videos/terminate_15_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### **Reward #3 : pole_pos** (Primary task: keep pole upright)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting pole_pos (Reward term) </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/pole_pose_0.png" alt="Image 1">
            <img src="images/pole_pose_1.png" alt="Image 2">
            <img src="images/pole_pose_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent pole_pos weight = -0.5, Right represent ploe_pos weight = -1.5 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/pole_pose_05_2000.mp4" controls></video>
            <video src="videos/pole_pose_15_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/pole_pose_05_8000.mp4" controls></video>
            <video src="videos/pole_pose_15_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/pole_pose_05_14000.mp4" controls></video>
            <video src="videos/pole_pose_15_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### **Reward #4 : cart_vel** (Shaping tasks: lower cart velocity)


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting cart_vel (Reward term) </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/cart_vel_0.png" alt="Image 1">
            <img src="images/cart_vel_1.png" alt="Image 2">
            <img src="images/cart_vel_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent cart_vel weight = -0.005, Right represent cart_vel weight = -0.015 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/cart_vel_0005_2000.mp4" controls></video>
            <video src="videos/cart_vel_0015_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/cart_vel_0005_8000.mp4" controls></video>
            <video src="videos/cart_vel_0015_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/cart_vel_0005_14000.mp4" controls></video>
            <video src="videos/cart_vel_0015_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### **Reward #5 : pole_vel** (Shaping tasks: lower pole angular velocity)


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting pole_vel (Reward term) </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/pole_vel_0.png" alt="Image 1">
            <img src="images/pole_vel_1.png" alt="Image 2">
            <img src="images/pole_vel_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent pole_vel weight = -0.006, Right represent pole_vel weight = -0.004 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/pole_vel_0006_2000.mp4" controls></video>
            <video src="videos/pole_vel_0004_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/pole_vel_0006_8000.mp4" controls></video>
            <video src="videos/pole_vel_0004_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/pole_vel_0006_14000.mp4" controls></video>
            <video src="videos/pole_vel_0004_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### Adjusting **boundary (termination term)** vs default configuration

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting Cart's boundary </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/boundary_0.png" alt="Image 1">
            <img src="images/boundary_1.png" alt="Image 2">
            <img src="images/boundary_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent bounds (-2,2), Right represent bounds (-4,4) </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/boundary_2_2000.mp4" controls></video>
            <video src="videos/boundary_4_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/boundary_2_8000.mp4" controls></video>
            <video src="videos/boundary_4_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/boundary_2_14000.mp4" controls></video>
            <video src="videos/boundary_4_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### Adjusting **joint_effort scale (Action space)** vs default configuration

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting Joint_effort scale </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/effort_0.png" alt="Image 1">
            <img src="images/effort_1.png" alt="Image 2">
            <img src="images/effort_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent joint_effort scale = 50, Right represent joint_effort scale = 150 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 2000</div>
            <video src="videos/effort_50_2000.mp4" controls></video>
            <video src="videos/effort_150_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 8000</div>
            <video src="videos/effort_50_8000.mp4" controls></video>
            <video src="videos/effort_150_8000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 14000</div>
            <video src="videos/effort_50_14000.mp4" controls></video>
            <video src="videos/effort_150_14000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


#### Adjusting **n Agent** vs default configuration

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Images and Videos with 3 Rows</title>
    <style>
        /* Style for the entire container */
        .container {
            max-width: 800px; /* Limit the overall width */
            margin: auto; /* Center the content */
            display: flex;
            flex-direction: column; /* Stack sections vertically */
            gap: 20px; /* Space between sections */
        }
        /* Style for the image section */
        .image-section {
            display: flex;
            flex-direction: column; /* Stack images vertically */
            gap: 10px; /* Space between images */
        }
        .image-section img {
            width: 100%; /* Make images responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Style for the video section */
        .video-section {
            display: grid;
            grid-template-columns: auto 1fr 1fr; /* Add a column for labels and 2 for videos */
            gap: 10px; /* Space between videos and labels */
        }
        /* Style for row labels */
        .video-label {
            display: flex;
            align-items: center; /* Center the label vertically */
            justify-content: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 16px; /* Adjust font size */
        }
        /* Style for videos */
        .video-section video {
            width: 100%; /* Make videos responsive */
            height: auto; /* Keep aspect ratio */
            border: 2px solid #ddd; /* Optional: Add a border */
            border-radius: 8px; /* Optional: Add rounded corners */
        }
        /* Top label for video section */
        .video-top-label {
            text-align: center; /* Center the label horizontally */
            font-weight: bold; /* Bold text */
            font-size: 18px; /* Increase font size for visibility */
            margin-bottom: 10px; /* Add space between the label and the video grid */
        }
    </style>
</head>
<body>
    <h4 style="text-align: center; font-weight: bold;">Graph & video for adjusting n_agent </h4>
    <div class="container">
        <!-- First Section: Images -->
        <div class="image-section">
            <img src="images/n_agent_0.png" alt="Image 1">
            <img src="images/n_agent_1.png" alt="Image 2">
            <img src="images/n_agent_2.png" alt="Image 3">
        </div>
        <!-- Horizontal Top Label -->
        <div class="video-top-label">Left represent n_agent = 32, Right represent n_agent = 128 </div>
        <!-- Second Section: Videos -->
        <div class="video-section">
            <!-- Row 1 Label -->
            <div class="video-label">step 4000 / 2000</div>
            <video src="videos/agent_32_4000.mp4" controls></video>
            <video src="videos/agent_128_2000.mp4" controls></video>
            <!-- Row 2 Label -->
            <div class="video-label">step 18000 / 4000</div>
            <video src="videos/agent_32_18000.mp4" controls></video>
            <video src="videos/agent_128_4000.mp4" controls></video>
            <!-- Row 3 Label -->
            <div class="video-label">step 30000 / 6000</div>
            <video src="videos/agent_32_30000.mp4" controls></video>
            <video src="videos/agent_128_6000.mp4" controls></video>
        </div>
    </div>
</body>
</html>


## **Part 3**  
Connect pratical experience with theoretical RL knowledge

<img src="images/RL.png" alt="Sample Image" width="500" style="display: block; margin: 0 auto;">


#### 3.1. What is reinforcement learning and its components according to your understanding? Giving examples of each component according to the diagram consider the `Cartpole` problem.

#### 3.2. What is the difference between the reward, return, and the value function?

#### 3.3. Consider policy, state, value function, and model as mathematical functions, what would each one take as input and output?